In [1]:
import numpy as np
from numpy import array
from pandas import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
test =  pd.read_csv('C:\\Users\\Ing\\Desktop\\project2.csv')
train = pd.read_csv('C:\\Users\\Ing\\Desktop\\project1.csv')

In [3]:
df = pd.concat((train, test))
dataset1 = df.drop(df.columns[[0, 3, 8,10]], axis=1)
dataset2 = pd.get_dummies(dataset1, columns=['Sex'])
dataset = pd.get_dummies(dataset2, columns=['Embarked'])
X = dataset.iloc[:, 1:12].values
y = dataset.iloc[:, 0].values

In [4]:
#missing DATA
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(X[:, 1:12])
X[:, 1:12] = imputer.transform(X[:, 1:12])

In [5]:
#split data into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3,random_state = 0)

In [6]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

X_train[7]

array([ 0.82870399, -2.04703355,  2.31232727,  1.88691806, -0.08642879,
       -0.72973501,  0.72973501, -0.50644264, -0.31782086,  0.64822484])

In [7]:
X_train= X_train.reshape(872,10,1)

In [8]:
y_train= y_train.reshape(872,1,1)

In [9]:
import tensorflow as tf
print ('Tensorflow version: ', tf.__version__)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras
print ('Keras version: ', keras.__version__)

Tensorflow version:  1.13.1
Keras version:  2.3.1


In [10]:
from keras.models import load_model
from keras.layers import Dense, Dropout, LSTM, Flatten, TimeDistributed, Input, GRU
from keras.models import Sequential, Model
from keras import optimizers
from keras.regularizers import l2
#import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt

In [11]:
idrop = 0
rdrop = 0.01
odrop = 0.01
epochs = 100
weight_decay = 1e-5

In [12]:
inputs = Input(shape=(10, 1))
output_step_ahead=1
RNN_model = LSTM(100, return_sequences = True, activation = 'relu', kernel_regularizer = l2(weight_decay),
                     recurrent_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay),\
                     dropout=idrop, recurrent_dropout=rdrop)(inputs)
RNN_model = Dropout(odrop)(RNN_model)
RNN_model = LSTM(100, return_sequences = True, activation = 'relu', kernel_regularizer = l2(weight_decay),
                     recurrent_regularizer=l2(weight_decay),bias_regularizer=l2(weight_decay),\
                     dropout=idrop, recurrent_dropout=rdrop)(inputs)
RNN_model = Dropout(odrop)(RNN_model)
RNN_model = LSTM(100, return_sequences = False, activation = 'relu', kernel_regularizer = l2(weight_decay), 
                    recurrent_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay),\
                    dropout=idrop, recurrent_dropout=rdrop)(RNN_model)
RNN_model = Dropout(odrop)(RNN_model)
RNN_model = keras.layers.Reshape(target_shape = (1, 100))(RNN_model)
RNN_model = Flatten()(RNN_model)
## prediction
pred_y = Dense(units = output_step_ahead, activation='linear', kernel_regularizer=l2(weight_decay),
                            bias_regularizer=l2(weight_decay))(RNN_model)
# Training of LSTM
RNN_model = Model(inputs=[inputs], outputs=[pred_y])
RNN_model.compile(loss="mse", optimizer=optimizers.adam(lr=0.001))
#LSTM training
RNN_hist = RNN_model.fit(X_train, y_train[:,:,0], shuffle = True, \
                             validation_split=0.1, verbose=0, epochs = epochs, batch_size = 32)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
X_test= X_test.reshape(437,10,1)
trainPredict1 = RNN_model.predict(X_test)
np.shape(trainPredict1)

(437, 1)

In [14]:

trainPredict = RNN_model.predict(X_train)
np.shape(trainPredict)

(872, 1)

In [16]:
m = trainPredict[:,0].shape
q = np.zeros(m, dtype = np.int)
j=0
for i in range(len(trainPredict)):
        if trainPredict[i] > 0.5:
            q[i] = 1
        else:
            q[i] = 0

In [17]:
print("Accuracy: "  + str(np.mean((q[:] == y_train[:]))))

Accuracy: 0.5428204696574362


In [97]:
m = trainPredict1[:,0].shape
p = np.zeros(m, dtype = np.int)
j=0
for i in range(len(trainPredict1)):
        if trainPredict1[i] > 0.5:
            p[i] = 1
        else:
            p[i] = 0

In [98]:
print("Accuracy: "  + str(np.mean((p[:] == y_test[:]))))

Accuracy: 0.8581235697940504
